In [2]:
# https://practicaldatascience.co.uk/machine-learning/how-to-tune-a-lightgbmclassifier-model-with-optuna

import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import optuna
from optuna.samplers import TPESampler
import pickle

In [3]:
X, y = load_wine(return_X_y=True, as_frame=True)
X.sample(5)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
86,12.16,1.61,2.31,22.8,90.0,1.78,1.69,0.43,1.56,2.45,1.33,2.26,495.0
161,13.69,3.26,2.54,20.0,107.0,1.83,0.56,0.50,0.80,5.88,0.96,1.82,680.0
14,14.38,1.87,2.38,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547.0
118,12.77,3.43,1.98,16.0,80.0,1.63,1.25,0.43,0.83,3.40,0.70,2.12,372.0


In [4]:
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
base_model = lgb.LGBMClassifier()
base_model.fit(X_train, y_train)


LGBMClassifier()

In [6]:
y_pred = base_model.predict(X_test)
y_pred


array([2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 0, 0, 1,
       2, 0, 0, 2, 0, 0, 0, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0,
       0, 0, 1, 0, 0, 0, 1, 2, 2, 0])

In [7]:
accuracy_score(y_test, y_pred)

0.9814814814814815

In [8]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        23
           1       1.00      0.95      0.97        19
           2       1.00      1.00      1.00        12

    accuracy                           0.98        54
   macro avg       0.99      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



In [9]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy


In [10]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)


[I 2023-01-24 09:33:52,310] A new study created in memory with name: lightgbm


[LightGBM] [Warning] bagging_fraction is set=0.48805353449026784, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48805353449026784
[LightGBM] [Warning] lambda_l1 is set=5.6649755830282306e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.6649755830282306e-05
[LightGBM] [Warning] feature_fraction is set=0.5813995435791038, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5813995435791038
[LightGBM] [Warning] lambda_l2 is set=0.030403280126677572, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.030403280126677572
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-01-24 09:33:53,739] Trial 0 finished with value: 0.9814814814814815 and parameters: {'lambda_l1': 5.6649755830282306e-05, 'lambda_l2': 0.030403280126677572, 'num_leaves': 2, 'feature_fraction': 0.5813995435791038, 'bagging_fraction': 0.48805353449026784, 'bagging_freq': 1, 'min_child_samples': 22}. Best is trial 0 with value: 0.9814814814814815.


[LightGBM] [Warning] bagging_fraction is set=0.8111317002380557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8111317002380557
[LightGBM] [Warning] lambda_l1 is set=1.2883876209377052e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2883876209377052e-05
[LightGBM] [Warning] feature_fraction is set=0.6515167086419769, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6515167086419769
[LightGBM] [Warning] lambda_l2 is set=3.72312200494449e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.72312200494449e-05
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


[I 2023-01-24 09:33:54,034] Trial 1 finished with value: 0.35185185185185186 and parameters: {'lambda_l1': 1.2883876209377052e-05, 'lambda_l2': 3.72312200494449e-05, 'num_leaves': 139, 'feature_fraction': 0.6515167086419769, 'bagging_fraction': 0.8111317002380557, 'bagging_freq': 2, 'min_child_samples': 89}. Best is trial 0 with value: 0.9814814814814815.


...

[I 2023-01-24 09:35:11,419] Trial 97 finished with value: 0.9814814814814815 and parameters: {'lambda_l1': 0.6079498001334874, 'lambda_l2': 8.734157518892662e-05, 'num_leaves': 155, 'feature_fraction': 0.7653890054087347, 'bagging_fraction': 0.8735975729697584, 'bagging_freq': 2, 'min_child_samples': 14}. Best is trial 14 with value: 1.0.


[LightGBM] [Warning] bagging_fraction is set=0.8441037792054307, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8441037792054307
[LightGBM] [Warning] lambda_l1 is set=3.76619506672726, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.76619506672726
[LightGBM] [Warning] feature_fraction is set=0.709106210808947, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.709106210808947
[LightGBM] [Warning] lambda_l2 is set=0.0006495313720384984, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0006495313720384984
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


[I 2023-01-24 09:35:12,839] Trial 98 finished with value: 0.9814814814814815 and parameters: {'lambda_l1': 3.76619506672726, 'lambda_l2': 0.0006495313720384984, 'num_leaves': 101, 'feature_fraction': 0.709106210808947, 'bagging_fraction': 0.8441037792054307, 'bagging_freq': 2, 'min_child_samples': 34}. Best is trial 14 with value: 1.0.


[LightGBM] [Warning] bagging_fraction is set=0.8608740398233584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8608740398233584
[LightGBM] [Warning] lambda_l1 is set=2.279186925966232, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.279186925966232
[LightGBM] [Warning] feature_fraction is set=0.6945612898897333, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6945612898897333
[LightGBM] [Warning] lambda_l2 is set=0.008309692873727997, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.008309692873727997
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


[I 2023-01-24 09:35:13,521] Trial 99 finished with value: 0.9814814814814815 and parameters: {'lambda_l1': 2.279186925966232, 'lambda_l2': 0.008309692873727997, 'num_leaves': 114, 'feature_fraction': 0.6945612898897333, 'bagging_fraction': 0.8608740398233584, 'bagging_freq': 5, 'min_child_samples': 59}. Best is trial 14 with value: 1.0.


In [11]:
print('Best parameters:', study.best_params)

Best parameters: {'lambda_l1': 9.81855410815485, 'lambda_l2': 5.0988035718136486e-06, 'num_leaves': 4, 'feature_fraction': 0.8844458592179963, 'bagging_fraction': 0.8661563398409448, 'bagging_freq': 2, 'min_child_samples': 17}


In [12]:
print('Best value:', study.best_value)

Best value: 1.0


In [13]:
print('Best trial:', study.best_trial)

Best trial: FrozenTrial(number=14, state=TrialState.COMPLETE, values=[1.0], datetime_start=datetime.datetime(2023, 1, 24, 9, 33, 59, 641529), datetime_complete=datetime.datetime(2023, 1, 24, 9, 34, 0, 219767), params={'lambda_l1': 9.81855410815485, 'lambda_l2': 5.0988035718136486e-06, 'num_leaves': 4, 'feature_fraction': 0.8844458592179963, 'bagging_fraction': 0.8661563398409448, 'bagging_freq': 2, 'min_child_samples': 17}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'min_child_samples': IntDistribution(high=100, log=False,

In [14]:
model = lgb.LGBMClassifier(**study.best_params)
model.fit(X_train, y_train)


[LightGBM] [Warning] bagging_fraction is set=0.8661563398409448, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8661563398409448
[LightGBM] [Warning] lambda_l1 is set=9.81855410815485, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.81855410815485
[LightGBM] [Warning] feature_fraction is set=0.8844458592179963, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8844458592179963
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=5.0988035718136486e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.0988035718136486e-06


LGBMClassifier(bagging_fraction=0.8661563398409448, bagging_freq=2,
               feature_fraction=0.8844458592179963, lambda_l1=9.81855410815485,
               lambda_l2=5.0988035718136486e-06, min_child_samples=17,
               num_leaves=4)

In [15]:
y_pred = model.predict(X_test)

In [16]:
accuracy_score(y_test, y_pred)

1.0

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       1.00      1.00      1.00        12

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

